In [ ]:
import re

#網址判斷   
def RegexMatchingTest(regex, input_text):
    pattern = re.compile(regex)
    result = re.search(pattern, input_text)

    if result:
        if result.lastindex is not None:
            for i in range(0, result.lastindex+1):
                return result.group(2)
    else:
        return None
regex = "(\w+):\/\/([^/:]+)?([^# !>]*)"     

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import re 

browser = webdriver.Chrome(
    executable_path='E:\Project\\1st-PyCrawlerMarathon\chromedriver\chromedriver.exe')
# 目標文章數量
target_count = 5

article_count = 1
article_url = []
article_content = {}
keepGoing = True
url = "https://www.cupoy.com/newsfeed/topicgrp/life_tw"

browser.get(url)
time.sleep(5)

while keepGoing:
    html = browser.page_source
    soup = BeautifulSoup(html, "lxml")

    news_block = soup.find_all('div', class_='sc-eEieub sc-iuDHTM ibJqYc')

    for news_item in news_block:
        url = news_item.find(
            'a', class_='sc-RbTVP sc-kEmuub eSJXii').get("href")
        article_content.setdefault(article_count, {})
        # 目標文章數量-1篇之前處理
        if url not in article_url and article_count < target_count:
            article_url.append(url)
            article_content[article_count]['標題'] = news_item.find('h6').string
            article_content[article_count]['分類'] = news_item.find(
                'div', class_='sc-gacfCG bPSpUf').string
            article_content[article_count]['摘要'] = news_item.find(
                'p', class_='sc-FQuPU sc-ciodno bvnzOw').string
            article_content[article_count]['連結'] = url               
            article_content[article_count]['網域'] = RegexMatchingTest(regex, url)   
            article_count += 1
        # 目標文章數量最後一篇處理
        elif url not in article_url and article_count == target_count:
            article_content[article_count]['標題'] = news_item.find('h6').string
            article_content[article_count]['分類'] = news_item.find(
                'div', class_='sc-gacfCG bPSpUf').string
            article_content[article_count]['摘要'] = news_item.find(
                'p', class_='sc-FQuPU sc-ciodno bvnzOw').string
            article_content[article_count]['連結'] = url
            article_content[article_count]['網域'] = RegexMatchingTest(regex, url)
            keepGoing = False

    time.sleep(2)
    browser.execute_script("window.scrollBy(0,10000)")

# print(article_content)
browser.quit()
frame = pd.DataFrame(article_content)
frame.T


,標題,分類,摘要,連結,網域
1,和失智長輩一起去旅行！六福旅遊的「喘息服務」，如何突破框架、打造新的生意？ | 經理人,旅遊,創造一項新服務，僅從服務對象身上看到未被滿足的需求並不夠，因為使用者往往不知道自己需要什麼；...,https://www.managertoday.com.tw/columns/view/5...,www.managertoday.com.tw
2,Google用AI自動重新剪輯影片 | iThome,攝影,Google以人工智慧技術重新剪裁16:9或是4:3長寬比的影片，讓影片更適合在各種螢幕大小...,https://www.ithome.com.tw/news/135833,www.ithome.com.tw
3,特斯拉召回至少 1.8 萬臺 Model X，轉向器有失去動力的安全疑慮 | MoneyDJ...,汽車,召回的方案適用於 2016 年生產的 Model X，在 2016 年 10 月中旬之後製造...,https://blog.moneydj.com/news/2020/02/15/%e7%8...,blog.moneydj.com
4,窮人的高速公路不只省錢 濕地、夕陽、燈塔、美食一次體驗 | 信傳媒,美食,交通部公路總局與天下雜誌合作出版《吹海風的路:臺61線西部濱海快速公路》一書，1,https://www.cmmedia.com.tw/home/articles/19923,www.cmmedia.com.tw
5,菲律賓解除旅遊禁令關鍵：在臺菲律賓移工震怒｜天下雜誌,旅遊,菲律賓政策在14日急轉彎，解除在10日對臺頒布的旅遊禁令。菲律賓詢問者日報（Philippi...,https://www.cw.com.tw/article/article.action?i...,www.cw.com.tw
